In [ ]:
# -*- coding: utf-8 -*-   
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn import linear_model
import plotly.plotly as py
import plotly.graph_objs as go
import re
from scipy import stats

In [ ]:
from matplotlib.font_manager import FontProperties
font = FontProperties(fname=r"c:\windows\fonts\SimSun.ttc", size=14)

In [ ]:
db = pd.read_excel(r'C:\Users\ren.x.1\Desktop\VS_Pricing\JFM_HC_ePOS.xlsx',encoding='gbk')
db.shape

In [ ]:
db.dropna(inplace = True)

In [ ]:
db.BrandName.value_counts()

In [ ]:
# df = db[db.BrandName == u'沙宣（VS）']
df = db.copy()

In [ ]:
df.Abnormal.value_counts()

In [ ]:
# exclude coupon,promotion pack, abnormal wholesale
df = df[- df.Mechanism.str.contains(u"领券")]
df = df[df.Mechanism != 'PP']
df = df[df.Abnormal == 'normal']

In [ ]:
df.Abnormal.value_counts()

In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
def jointplt(ProductCode):
    SKU = df[df['ProductCode'] == ProductCode][['Sales_Price',u'Sales_Num','uv']]
    #SKU = SKU.groupby(by = 'Sales_Price',as_index=False).mean()
    #SKU = SKU[SKU.Sales_Num >= 10]
    SKU['Productivity'] = np.log(SKU.Sales_Num/SKU.uv)
    SKU.Sales_Price = np.log(SKU[u'Sales_Price'])
    sns.jointplot(x="Sales_Price", y="Productivity", kind = 'reg', color = 'r', data=SKU)
    #plt.ylim(-4,0)
    #plt.xlim(3,5)
    #plt.title(SKU_NAME,fontproperties=font)

In [ ]:
def statsReg(ProductCode):
    SKU = df[df.ProductCode == ProductCode]
    x = np.log(SKU.Sales_Price/SKU.uv)
    y = np.log(SKU.Sales_Num)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    if p_value < 0.05:
        return np.round(slope,decimals=1)
    else:
        return np.nan

In [ ]:
PE_Hash_1 = {}
for code in df.ProductCode:
    PE_Hash_1[code] = statsReg(code)

PE_DF_1 =  pd.DataFrame.from_dict(PE_Hash_1, orient='index', dtype=None)
PE_DF_1.columns = ['PE']
PE_DF_1.head()

In [ ]:
PE_DF_1.PE.hist(bins = 20)

In [ ]:
prodNames = []
for idx in PE_DF_1.index:
    prodNames.append(df[df.ProductCode == idx].ProductName.iloc[0])
PE_DF_1['productName'] = prodNames

margin = []
for idx in PE_DF_1.index:
    margin.append(np.round(float(df[df.ProductCode == idx].Offtake.sum())/df[df.ProductCode == idx].Total_Cost.sum(),decimals=2) - 1)
PE_DF_1['margin'] = margin

averagePrice = []
for idx in PE_DF_1.index:
    averagePrice.append(np.round(float(df[df.ProductCode == idx].Offtake.sum())/df[df.ProductCode == idx].Sales_Num.sum(),decimals=2))
PE_DF_1['averagePrice'] = averagePrice

cost = []
for idx in PE_DF_1.index:
    cost.append(df[df.ProductCode == idx].Unit_Cost.iloc[0])
PE_DF_1['cost'] = cost

# calculate the SRP maximize profit
SRP = []
for idx in PE_DF_1.index:
    # quadratic function symmetric axis
    if PE_DF_1.loc[idx].PE < 0:
        SRP.append(np.round(((1.0/PE_DF_1.loc[idx].PE + 1 - float(PE_DF_1.loc[idx].cost)/PE_DF_1.loc[idx].averagePrice)*-0.5 + 1)*\
        PE_DF_1.loc[idx].averagePrice,decimals = 2))
    else:
        SRP.append(np.nan)
PE_DF_1['SuggestedPrice'] = SRP

PE_DF_1['priceAdjust%'] = np.round((PE_DF_1['SuggestedPrice']/PE_DF_1['averagePrice'] - 1),decimals=2)*100

In [ ]:
PE_DF_1['priceAdjust%'].describe()

In [ ]:
PE_DF_1['priceAdjust%'].hist(bins = 40)
plt.title('Price Adjustment% Histogram')

In [ ]:
PE_DF_1.describe()

In [ ]:
PE_DF_1.PE.isnull().value_counts()

In [ ]:
# suggested price adjust SKUs
mask = (PE_DF_1.PE > -1) & (PE_DF_1.margin < - 0.1)
PE_DF_1[mask]

In [ ]:
jointplt(1265447)

**Per Mechanism**

In [ ]:
def statsRegPerMech(ProductCode):
    SKU = df[df.ProductCode == ProductCode]
    PEs = []
    for mech in df.Mechanism.unique():
        SKU_PerMech = SKU[SKU.Mechanism == mech]
        if SKU_PerMech.empty:
            PEs.append('NODATA')
        else:
            x = np.log(SKU_PerMech.Sales_Price)
            y = np.log(SKU_PerMech.Sales_Num/SKU_PerMech.uv)
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            if p_value < 0.05:
                PEs.append(np.round(slope,decimals=1))
            else:
                PEs.append('NONLINEAR')
    return PEs

In [ ]:
PE_Hash = {}
for code in df.ProductCode:
    PE_Hash[code] = statsRegPerMech(code)

PE_DF =  pd.DataFrame.from_dict(PE_Hash, orient='index', dtype=None)
PE_DF.columns = df.Mechanism.unique()
PE_DF.head()

**Three-Chanel split: TPR, 199-100, others**

In [ ]:
def statsByMechReg(ProductCode,df):
    SKU = df[df.ProductCode == ProductCode]
    x = np.log(SKU.Sales_Price)
    y = np.log(SKU.Sales_Num)/SKU.uv)
    print y
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    if p_value < 0.05:
        return np.round(slope,decimals=1)
    else:
        return 'Mark'

In [ ]:
df.Mechanism.value_counts()

In [ ]:
df1 = df[df.Mechanism == 'single TPR']

In [ ]:
df2 = df[df.Mechanism == u'满199减100']

In [ ]:
mask3 = (df.Mechanism != 'single TPR') & (df.Mechanism != u'满199减100')

In [ ]:
df3 = df[mask3]

In [ ]:
df3.Mechanism.value_counts()

In [ ]:
PE_1 = {}
for code in df1.ProductCode:
    PE_1[code] = statsByMechReg(code,df1)

PE_DF1 =  pd.DataFrame.from_dict(PE_1, orient='index')
PE_DF1.columns = ['TPR_PE']

In [ ]:
range1 = []
for idx in PE_DF1.index:
    minnimum = df1[df1.ProductCode == idx].Sales_Price.min()
    maximum = df1[df1.ProductCode == idx].Sales_Price.max()
    if minnimum != maximum:
        strRange = '[' + np.str(np.round(minnimum,decimals = 2)) +',' + np.str(np.round(maximum,decimals = 2)) + ']'
        range1.append(strRange)
    else:
        range1.append(np.str(np.round(minnimum,decimals = 2)))

PE_DF1['TPR_Range'] = range1

In [ ]:
PE_DF1.head()

In [ ]:
PE_2 = {}
for code in df2.ProductCode:
    PE_2[code] = statsByMechReg(code,df2)

PE_DF2 =  pd.DataFrame.from_dict(PE_2, orient='index')
PE_DF2.columns =['199-100_PE']
PE_DF2.head()

In [ ]:
range2 = []
for idx in PE_DF2.index:
    minnimum = df2[df2.ProductCode == idx].Sales_Price.min()
    maximum = df2[df2.ProductCode == idx].Sales_Price.max()
    if minnimum != maximum:
        strRange = '[' + np.str(np.round(minnimum,decimals = 2)) +',' + np.str(np.round(maximum,decimals = 2)) + ']'
        range2.append(strRange)
    else:
        range2.append(np.str(np.round(minnimum,decimals = 2)))

PE_DF2['199-100_Range'] = range2

In [ ]:
PE_DF2

In [ ]:
PE_3 = {}
for code in df3.ProductCode:
    PE_3[code] = statsByMechReg(code,df3)

PE_DF3 =  pd.DataFrame.from_dict(PE_3, orient='index')
PE_DF3.columns =['Others_PE']
PE_DF3.head()

In [ ]:
range3 = []
for idx in PE_DF3.index:
    minnimum = df3[df3.ProductCode == idx].Sales_Price.min()
    maximum = df3[df3.ProductCode == idx].Sales_Price.max()
    if minnimum != maximum:
        strRange = '[' + np.str(np.round(minnimum,decimals = 2)) +',' + np.str(np.round(maximum,decimals = 2)) + ']'
        range3.append(strRange)
    else:
        range3.append(np.str(np.round(minnimum,decimals = 2)))

PE_DF3['Others_Range'] = range3

In [ ]:
PE_ByMech_1 = PE_DF1.join(PE_DF2,how='outer')

PE_ByMech = PE_ByMech_1.join(PE_DF3,how='outer')

In [ ]:
PE_ByMech.head()

In [ ]:
output = pd.merge(db, PE_ByMech, how='left',left_on=['ProductCode'],right_index = True)
output.to_excel('output0509.xlsx')

In [ ]:
prodNames = []
for idx in PE_ByMech.index:
    prodNames.append(df[df.ProductCode == idx].ProductName.iloc[0])
PE_ByMech['productName'] = prodNames

margin = []
for idx in PE_ByMech.index:
    margin.append(np.round(float(df[df.ProductCode == idx].Offtake.sum())/df[df.ProductCode == idx].Total_Cost.sum(),decimals=2) - 1)
PE_ByMech['margin'] = margin

averagePrice = []
for idx in PE_ByMech.index:
    averagePrice.append(np.round(float(df[df.ProductCode == idx].Offtake.sum())/df[df.ProductCode == idx].Sales_Num.sum(),decimals=2))
PE_ByMech['averagePrice'] = averagePrice

cost = []
for idx in PE_ByMech.index:
    cost.append(df[df.ProductCode == idx].Unit_Cost.iloc[0])
PE_ByMech['cost'] = cost

In [ ]:
PE_ByMech.head()

In [ ]:
PE_ByMech.to_excel('peByMechanism.xlsx')